# Analyse FinBERT using LIT

To analyse the FinBERT model using the Language Interpretability Tool (LIT), you would need to integrate the model into LIT's environment, allowing you to interactively explore its behavior, interpret predictions, and test for fairness, robustness, and other aspects.

In [1]:
from lit_nlp.api import model as lit_model
from lit_nlp.api import dataset as lit_dataset
from lit_nlp.api import types as lit_types
from lit_nlp import notebook
from transformers import BertTokenizer, BertForSequenceClassification, BertConfig, pipeline
import pandas as pd
import torch

You'll need to load the FinBERT model within a custom model class that LIT can recognize.

In [2]:
class FinBERTModel(lit_model.Model):
    """A wrapper for FinBERT to work with LIT."""
    
    def __init__(self):
        # Load FinBERT model and tokenizer
        self.model_name = "yiyanghkust/finbert-tone"
        self.model = BertForSequenceClassification.from_pretrained(self.model_name, num_labels=3)
        self.tokenizer = BertTokenizer.from_pretrained(self.model_name)
        self.config = BertConfig.from_pretrained(self.model_name)

    def _load_model(self):
        return BertForSequenceClassification.from_pretrained(self.model_name, num_labels=3)
    
    def _load_tokenizer(self):
        return BertTokenizer.from_pretrained(self.model_name)
        
    def input_spec(self) -> lit_types.Spec:
        return {
            "text": lit_types.TextSegment()
        }
    
    def output_spec(self) -> lit_types.Spec:
        return {
            "score": lit_types.MulticlassPreds(vocab=["Positive", "Neutral", "Negative"], parent="label"),
            "label": lit_types.CategoryLabel(vocab=["Positive", "Neutral", "Negative"]),
        }
    
    def predict(self, inputs):
        # create a list of strings from the input
        # input_list = []
        results = []
        for input in inputs:
            # input_list.append(input["text"])
            
            # if you don't want to use the pipeline, you can use the model directly
            with torch.no_grad():
                encoded_input = self.tokenizer(input["text"], padding=True, return_tensors='pt')
                output = self.model(**encoded_input)
                probs = torch.softmax(output['logits'], dim=1)
                label = self.config.id2label[torch.argmax(probs).item()]
                results.append({
                    "score": probs[0].tolist(),
                    "label": label
                })

        # pipe = pipeline("text-classification", model=self.model, tokenizer=self.tokenizer)
        # results = pipe(input_list)
        return results

# Instantiate the FinBERT model
model = FinBERTModel()

/Users/asabuncuoglu/Documents/faid-test-financial-sentiment-analysis/.venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [3]:
class FinPhrasebankDataset(lit_dataset.Dataset):
  """Loader for MultiNLI development set."""

  TEXT_COLUMN = "text"
  TARGET_COLUMN = "sentiment"

  def __init__(self, path: str):
    df = pd.read_csv(path, names=[self.TARGET_COLUMN, self.TEXT_COLUMN], encoding="utf-8", encoding_errors="replace")
    # Store as a list of dicts, conforming to self.spec()
    df = df.head(20) # limit to 20 examples for speed
    self._examples = pd.DataFrame.to_dict(df, orient="records")

  def spec(self) -> lit_types.Spec:
    return {
      'text': lit_types.TextSegment(),
      'sentiment': lit_types.CategoryLabel(vocab=[self.TARGET_COLUMN, self.TEXT_COLUMN])
    }

After wrapping the FinBERT model, you need to launch the LIT server to interactively analyze the model.

In [ ]:
filename = "../data/external/financialphrasebank.csv"

# Initialize dataset and model
dataset = FinPhrasebankDataset(path=filename)
model = FinBERTModel()

# Create the LIT widget
lit_widget = notebook.LitWidget(models={"finbert": model}, datasets={"finphrasebank": dataset}, port=8890)
#lit_widget.render()

Once the server is running, you can access LIT in your web browser by navigating to `http://localhost:8890`. Or optionally, you can run it on your notebook:

```python
widget.render(height=600)
```

Note that, I could not run this widget on VSCode's Jupyter Notebook Editor, so I only tested the functionality on the browser. In the LIT interface, you can interactively explore the model's predictions, compare different inputs, visualize feature importance, and analyze model behavior through various interpretability techniques.

### Analyze Model Behavior
- **Examine Predictions**: Input different financial texts and observe how FinBERT classifies the sentiment. You can compare results across multiple texts or even multiple models if loaded.
- **Saliency Maps**: Use saliency maps to understand which words contribute most to the sentiment classification.
- **Counterfactuals**: Test counterfactual scenarios by modifying parts of the text to see how the model’s prediction changes, helping you understand its decision-making process.
- **Fairness Analysis**: Analyze how the model performs across different types of financial data (e.g., news articles about different industries or regions) to check for any biases.

This setup enables you to ensure the model’s fairness, understand its decision-making process, and refine its performance for financial sentiment analysis.